## Investigating Relationships Between The Backdrop of Provincial Economic Volatility and Wider Mental Health in Alberta:


##### Introduction
Our proposed digital exploration will analyze the linkage between economic factors and mental health trends within the wider Alberta population. The Mental Health Commission of Canada conducted an economic review in 2011 and provided an estimate of the impact of mental health illnesses on lost productivity due to absenteeism, presenteeism (present but less than fully productive at work) and turnover;  in 2011 alone, the cost to the economy was 6.3 billion dollars. This value is projected to rise to 16 billion dollars in 2041. In any given year, 1 in 5 Canadians experiences a mental illness or addiction problem and by the time Canadians reach 40 years of age, 1 in 2 have, or have had, a mental illness. This means that more than 6.7 million people in Canada are living with a mental health problem or illness today. That is 19.8% of Canada’s population in any given year. (*Why Investing In Mental Health Will Contribute To Canada’s Economic Prosperity and To the Sustainability of Our Healthcare System*, 2021, pp. 2-3) It is likely that because there is stigma attached to harbouring a mental health diagnosis, that reported metrics are understated.  According to the World Health Organization (WHO) the incidence of mental illness is expected to rise as economic drivers become increasingly dynamic, and the “gig economy” becomes more commonplace. (*Mental health action plan*, 2013, pp. 6-7) The collective provincial population would benefit vastly from applied data analytics in order to address the compounding mental health crisis that is ongoing within the province. It is possible that in the future, applied analytics will be able to guide policy makers to utilize provincially budgeted resources in a more targeted and efficient manner. (Smetanin et al, 2011, pp. 45 -55) 

On an aggregate level, it is well documented that the relationship between economic inequality and mental health exists, but despite this, a reductionist biomedical model assessing mental health on an individual and physiological basis has persisted within the academic medical community. This has limited the ability of corporate entities and policy makers to address inequalities within the mental health sphere. (Macintyre et al, 2018, p. 4)  Another driving factor of mental health inequality has been economic volatility. The province of Alberta has experienced significant economic hardship following the collapse of Western Canadian Select (WCS)  oil prices and NOVA/AECO-C gas prices in 2014. The new commodity price environment also spurred a wider thematic global investment shift away from the energy industry and mounted pressure on corporate entities to support ESG driven narratives.

##### Guiding Questions
Our analysis will look to capture the essence of economic reality which exists within municipalities and the wider province, and overlay that theme with mental health related data. The analysis will attempt to answer three core questions: 

- Is there an identifiable relationship between the dynamic economic situation of the province and the wider mental health trend within Alberta?
- Which subpopulations have been the most adversely affected by the economic volatility in Alberta within the last decade?
- What have been the trends in more granular hospital-based mental health data, and do those findings relate to the wider economic analysis and subgroup analysis?


In [24]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

import plotly.express as px
from plotly.subplots import make_subplots
import ipywidgets as widgets
from ipywidgets import interact
import plotly.graph_objs as go

The main data set we will be using is . This data set privides meaningful information about the self reported quality of mental health among various age groups across alberta. However, we need to adjust the format in order to compare the following report with those of the so that the years are presented as the independent variables with all information presented as independent variables. This required the transposition of the varaibles as well as cutting the precentage 

In [4]:
main_MH =pd.read_csv("CanadaMentalHealth.csv")
main_MH["Title"] = main_MH["Ag group"] + main_MH["Indicators"]
main_MH.drop(columns = ['Ag group', 'Indicators'], inplace = True)
main_MH = main_MH.transpose()
main_MH.rename(columns=dict(main_MH.iloc[-1,:]), inplace = True)
main_MH.drop(main_MH.tail(1).index,inplace=True)
tempDF = main_MH.iloc[6:12,]
tempDF.index = range(2015,2021)
tempDF.columns=['%Total+', '%Total-','%1217+', '%1217-','%1834+', '%1834-','%3549+', '%3549-', '%5064+', '%5064-', '%65+', '%65-']
main_MH = main_MH.drop(main_MH.tail(6).index)
main_MH['year'] = main_MH.index
tempDF['year'] = tempDF.index
tempDF['year'] = tempDF['year'].astype('int64')
main_MH['year'] = main_MH['year'].astype('int64')
main_MH = main_MH.merge(tempDF, on='year')
main_MH.index = range(2015,2021)

Next, we are interested in getting an annual report for the Activity Index of Alberts.

https://open.alberta.ca/opendata/alberta-activity-index-data-tables

In [5]:
AA = pd.read_excel("ActivityIndex.xlsx")
AA.dropna(axis=1, inplace=True)
AA['year'] = pd.DatetimeIndex(AA['Date']).year
AA['month'] = pd.DatetimeIndex(AA['Date']).month
AA = AA.groupby('year', as_index=False).aggregate('mean').drop(['month'], axis = 1)
AA.columns=['year', 'Mean AA']
AA = AA[AA['year'] >= 2015]
AA =  AA[AA['year'] < 2021]

main_MH = main_MH.merge(AA, on='year')
display(main_MH.head(1))

,"Total, 12 yars and ovrPrcivd mntal halth, vry good or xcllnt 6","Total, 12 yars and ovrPrcivd mntal halth, fair or poor 6","12 to 17 yarsPrcivd mntal halth, vry good or xcllnt 6","12 to 17 yarsPrcivd mntal halth, fair or poor 6","18 to 34 yarsPrcivd mntal halth, vry good or xcllnt 6","18 to 34 yarsPrcivd mntal halth, fair or poor 6","35 to 49 yarsPrcivd mntal halth, vry good or xcllnt 6","35 to 49 yarsPrcivd mntal halth, fair or poor 6","50 to 64 yarsPrcivd mntal halth, vry good or xcllnt 6","50 to 64 yarsPrcivd mntal halth, fair or poor 6",...,%1217-,%1834+,%1834-,%3549+,%3549-,%5064+,%5064-,%65+,%65-,Mean AA
0,2555000,210100,203000,8900,782600,70500,644900,54400,612000,55000,...,3.4,72.9,6.6,73.5,6.2,77.9,7.0,72.8,5.0,274.412729


In [29]:
fig = make_subplots(rows=2, cols=1)

fig.append_trace(go.Scatter(
    x=main_MH['year'],
    y=main_MH["18 to 34 yarsPrcivd mntal halth, fair or poor 6"],
    name = 'Mental Health'
), row=1, col=1)

fig.append_trace(go.Scatter(
    x=main_MH['year'],
    y=main_MH["Mean AA"],
    name = 'Alberta Economy'
), row=2, col=1)


fig.update_layout(height=600, width=600, title_text="Stacked Subplots")
fig.show()

In [15]:
testfig1.show()

In [10]:
## Hospital Data

MH = pd.read_excel("HMHDB_Mental_Health.xlsx", sheet_name="4 Combined LOS prov terr")
MH.rename(columns=dict(MH.loc[3,]), inplace = True)
MH = MH.iloc[4:46]
MH.dropna(axis=1, inplace = True)
MH.reset_index(drop = True, inplace = True)
MH['year'] = 2018
display(MH.head(1))

c:\users\jonpe\appdata\local\programs\python\python39\lib\site-packages\openpyxl\worksheet\_reader.py:312: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


,Hospital type,Province/territory,Median length of stay \n(days),Average length of stay \n(days),0.5% trimmed average \n(days),Total length of stay \n(days),year
0,General hospitals,Newfoundland and Labrador,6,14.22,12.93,26157,2018


Data taken form Alberta's economic dashboard allows us to view provincial wide economic indicators throughout the years. We are interested in being able to visualize the mass amount of information provided in the best substettable format as possible. Below, we used the plotly library in an effort to create interactive graphs that will allow us to view all economic indicator information that we need in an effort to understand major economic trends throughout various sectors.

In [11]:
## Economic Indicators

econDF = pd.DataFrame(columns=['When','Alberta', 'Sector'])

for files in os.listdir("Economic"):
    df = pd.read_csv("Economic/" + files)
    df['When'] = pd.to_datetime(df['When'])
    tempDF = df[["When", "Alberta"]] # Get When and Alberta columns
    if files != 'Population.csv':
        tempDF = tempDF.join(df.iloc[:,2], lsuffix='_left', rsuffix='_right') # Get second column
    else:
        tempDF['Pop'] = 'Population'
        
    tempDF.columns = ["When", "Alberta", "SubSector"]
    tempDF['Sector'] = files.split('.')[0]
    econDF = econDF.append(tempDF) # append to final df
    
econDF['Month'] = econDF['When'].dt.month
econDF['Day'] = econDF['When'].dt.day
econDF['Year'] = econDF['When'].dt.year

econDF['Sector2'] = econDF['SubSector'].astype(str) + " (" + econDF['Sector'].astype(str) + ")"

<ipython-input-11-898c7fe4a1b3>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tempDF['Pop'] = 'Population'
<ipython-input-11-898c7fe4a1b3>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tempDF['Sector'] = files.split('.')[0]


Using the economic indicators, we are interested in identifying any trends within our data that we could use in our comparison to trends likewise found within the mental health datasets are are provided. An interactive plot is used to identify these key trends on a year by year basis for each sector and subsector.

In [26]:
graphDF2 = econDF.groupby(['Year', 'Month', 'Sector']).mean().drop('Day', axis = 1)

# Now let's explore the data with the help of a drop-down interactor.

# Build a bar chart that we'll then update with the help of a call-back function
li = go.Scatter()
fig_line = go.FigureWidget(data=li)

month_labels = {1:'Jan',2:'Feb',3:'Mar',4:'Apr',5:'May', 6:'Jun', 7:'Jul', 8:'Aug', 9:'Sep', 10:'Oct', 11:'Nov', 12:'Dec'}

# A list passed to interact() will yield a drop-down interactor
@interact(sector = list(econDF['Sector'].unique()))

def update_bar(sector):
    
    graphDF2 = econDF[econDF['Sector'] == sector]
    graphDF2 = graphDF2.groupby(['Year', 'SubSector']).mean().drop(['Day', 'Month'], axis = 1)
    graphDF2.reset_index(inplace = True)
    data = graphDF2
    fig_line.update_traces(x=[max(data['Year']),max(data['Year'])],
                       y=[max(data['Alberta']),max(data['Alberta'])])
    
    for subsector in data['SubSector'].unique():
        data1 = data[data['SubSector'] == subsector]
        fig_line.add_trace(
            go.Scatter(x=data1['Year'],
                       y=data1['Alberta'],
                      name = subsector))
        
    fig_line.update_layout(title_text="{0} results".format(sector))
    fig_line.update_layout(showlegend=False)

fig_line

interactive(children=(Dropdown(description='sector', options=('CattlePrice', 'NetMigration', 'Population', 'Pr…

FigureWidget({
    'data': [{'type': 'scatter',
              'uid': '2ef8828d-a33e-480f-943b-d72b9852c988',
 …

Some key trends that we are able to indentify are the rise is cattle prices in 2015, the rise of Oil prices starting in 2008, and unemployment take a slow upturn starting in 2008, with another spike in 2016, then 2020. All of these are inline with the hypothesized dates for economic downturn that offered significant challanges for residents of Alberta. the rise in these specific indicators are the summation of key livelihood indicators for city residents, food, transportation, and living arrangments. The significant spikes in these sectors suggest that we would see increases in metnal health related hosptilizations and so on during these significant years 2018, 2015, and 2020.


Secondly, we are interested in visualizing each individual sub-sector on a month to month basis. This will allow us to backtrack to key dates in Alberta's economic history and track how differing sectors changed on a month to month basis. This will also allow us to visualize any interesting points our disperities that we notice from the above line plot for economic indicators.

In [27]:
graphDF = econDF.groupby(['Year', 'Month', 'Sector']).mean().drop('Day', axis = 1)

# Now let's explore the data with the help of a drop-down interactor.

# Build a bar chart that we'll then update with the help of a call-back function
bar = go.Bar()
fig_bar = go.FigureWidget(data=bar)
#fig_bar.update_yaxes(range=[0, 250])

month_labels = {1:'Jan',2:'Feb',3:'Mar',4:'Apr',5:'May', 6:'Jun', 7:'Jul', 8:'Aug', 9:'Sep', 10:'Oct', 11:'Nov', 12:'Dec'}

# A list passed to interact() will yield a drop-down interactor
@interact(year=[*range(max(econDF['Year']),min(econDF['Year']),-1)],
          sector = list(econDF['Sector2'].unique()))

def update_bar(sector, year=2019):
    graphDF =  econDF[econDF['Sector2'] == sector]
    graphDF = graphDF.groupby(['Year', 'Month']).mean().drop('Day', axis = 1)

    data = graphDF.loc[year].mean(axis=1,skipna=True)
    fig_bar.update_traces(x=pd.Series(data.index.values).values, #.map(month_labels).values,
                          y=data.values)
    fig_bar.update_layout(title_text="{0} results for {1}".format(sector, year))

fig_bar

interactive(children=(Dropdown(description='sector', options=('Slaughter, calves (CattlePrice)', 'Slaughter, c…

FigureWidget({
    'data': [{'type': 'bar',
              'uid': '89bb2ba1-bc10-431b-9384-4551d0713aa8',
     …

Looking back on the key economic indicators from above, we find that although we saw singificant spikes for unemployment in the years 2009 and 2016, there was no key contributing month of the year. Both years saw a steady incline in unemployment through the whole year. Meanhwile, 2020 saw a siginficant spike in April due to the Covid-19 pandemic. Oil prices saw a dramic rise in 2009 with prices of WTI almost doubling from 40$ to 80$, with a the prices never really dipping below 60$ after that point. and lastly, we find that the price of cattle slaughter likewise had a gradual gain, peaking in May at 193$